In [ ]:
import spacy
import os
import sys
import skweak
nlp = spacy.load("nl_core_news_md")

In [ ]:
import tarfile

# We retrieve the texts from Wablieft tarfile
texts = [] 
archive_file = tarfile.open("data/plain.tar.xz")
for archive_member in archive_file.getnames():
    if archive_member.endswith(".txt"):
        #weird encoding because of Turkish letters
        text = archive_file.extractfile(archive_member).read().decode("cp850")
        texts.append(text)

In [ ]:
docs = list(nlp.pipe(texts))

In [ ]:
def name_detector_fn(doc):
    names = ["David", "Cameron", "William", "Bos"]
    for token in doc: 
        if token.text in names:
            yield token.i, token.i+1, "PERSON"

name_detector = skweak.heuristics.FunctionAnnotator("name_detector", name_detector_fn)

def address_detector_fn(doc):
    for token in doc: 
        if token.text[0].isupper() and doc[token.i+1].is_digit:
            yield token.i, token.i+2, "LOCATION"

address_detector = skweak.heuristics.FunctionAnnotator("address_detector", address_detector_fn)

def company_detector_fn(doc):
    companies = ["Microsoft", "Apple", "Gemeente Amsterdam", "Universiteit van Amsterdam", "UvA"]
    for token in doc:
        if token.text in companies:
            yield token.i, token.i+1, "ORG"

company_detector = skweak.heuristics.FunctionAnnotator("company_detector", company_detector_fn)

names = skweak.gazetteers.extract_json_data("data/geonames.json", spacy_model="en_core_web_sm")
name_annotator = skweak.gazetteers.GazetteerAnnotator("locations", names)

In [ ]:
processed = list(name_detector.pipe(name_annotator.pipe(docs[:25])))
hmm = skweak.aggregation.HMM("hmm", ["PERSON", "LOCATION", "ORG", "GPE"])

#skweak.utils.display_entities(processed[7], "locations")
hmm.fit_and_aggregate(processed)
skweak.utils.display_entities(processed[7], "hmm", add_tooltip=True)

In [ ]:
for doc in processed:
    if "hmm" in doc.spans.keys():
        doc.ents = doc.spans["hmm"]
    else: 
        doc.ents = ()

docs = docs[:100]
skweak.utils.docbin_writer(docs, "data/wablieft.spacy")

In [ ]:
!spacy init fill-config base_config.cfg config.cfg

In [ ]:
!spacy train config.cfg --paths.train data/wablieft.spacy --paths.dev data/wablieft.spacy --initialize.vectors nl_core_news_md --output output

In [ ]:
model = spacy.load("output/model-best")

In [ ]:
doc = processed[7]

In [ ]:
skweak.utils.display_entities(model("Hoi ik ben William en ik woon in New York. Ik kom uit Flanders en ben geboren in Antwerpen."))

In [ ]:
from lison2020 import annotations
from spacy.tokens import DocBin
test = DocBin().from_disk("data/ned_testb.spacy")
sys.path.insert(0, './lison2020')

In [ ]:
annotator = annotations.FullAnnotator().add_all()

In [ ]:
import evaluation

In [ ]:
evaluation.spacy_benchmark("data/ned_testb.spacy")